# Regressão Logística em Python

Neste notebook apresentamos um caso prático de análise de regressão logística com dados reais. A análise descrita usa dados socioeconômicos da cidade de Curitiba para a compreensão dos fatores associados à urbanização nos bairros da cidade. Os dados foram coletados do [IPPUC](https://www.ippuc.org.br/) (Instituto de Pesquisa e Planejamento Urbano de Curitiba). A análise é baseada na biblioteca [StatsModels](https://www.statsmodels.org), que implementa uma grande gama de modelos de regressão.

In [35]:
# Importação de pacotes
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

## Lendo dados de um arquivo

Os dados coletados foram agregados em um arquivo CSV, exibido abaixo e disponibilizado no reposítório deste tutorial.

In [38]:
# lê o arquivo CSV
df = pd.read_csv('indices_curitiba.csv')
# mostra o conteúdo do DataFrame
df.head(3)

,Localidade,Renda_Média,Taxa_Analfabetismo,Área_Verde_m2_percapta,Área_Verde_km2,Estabelecimentos_Ativos_Indústria,Estabelecimentos_Ativos_Comércio,Estabelecimentos_Ativos_Serviço,População_Total,Área_hectare,Área_km2,Densidade_Demográfica_Hab_por_ha,Área_Verde_pct,Nível_Urbanização
0,Abranches,650.0,3.71,116.38,1.534982,106.0,373.0,387.0,13189.0,431.63,4.32,30.56,35.532000,Baixo
1,Ahú,1500.0,0.87,26.24,0.301921,147.0,548.0,727.0,11506.0,185.49,1.85,62.03,16.320048,Médio
2,Alto Boqueirão,650.0,3.36,61.88,3.321273,353.0,1421.0,749.0,53671.0,1198.73,11.99,44.77,27.700357,Baixo


## Tratando os dados

Nossa variável dependente será a urbanização do bairro (coluna Urbanizado), que conterá 1 quando o bairro for urbanizado e 0 quando não. Vamos considerar urbanizados os bairros com nível de urbanização alto e não urbanizados os de nível baixo. Também vamos criar uma variável de estabelecimentos de serviço por mil habitantes. O código destes procedimentos está abaixo:

In [39]:
df['Urbanizado'] = df['Nível_Urbanização'].map({'Baixo':0, 'Alto':1})
df = df.dropna(how='any')
df['Urbanizado'] = df['Urbanizado'].astype(int)
df['Estabelecimentos_Ativos_Serviço_per1k'] = 1000 * df['Estabelecimentos_Ativos_Serviço']/df['População_Total']
df.head()

,Localidade,Renda_Média,Taxa_Analfabetismo,Área_Verde_m2_percapta,Área_Verde_km2,Estabelecimentos_Ativos_Indústria,Estabelecimentos_Ativos_Comércio,Estabelecimentos_Ativos_Serviço,População_Total,Área_hectare,Área_km2,Densidade_Demográfica_Hab_por_ha,Área_Verde_pct,Nível_Urbanização,Urbanizado,Estabelecimentos_Ativos_Serviço_per1k
0,Abranches,650.0,3.71,116.38,1.534982,106.0,373.0,387.0,13189.0,431.63,4.32,30.56,35.532000,Baixo,0,29.342634
2,Alto Boqueirão,650.0,3.36,61.88,3.321273,353.0,1421.0,749.0,53671.0,1198.73,11.99,44.77,27.700357,Baixo,0,13.955395
6,Augusta,520.0,4.85,680.84,4.492184,30.0,57.0,32.0,6598.0,917.25,9.17,7.19,48.987832,Baixo,0,4.849955
9,Barreirinha,750.0,2.49,59.58,1.073521,196.0,572.0,408.0,18017.0,362.07,3.62,49.76,29.655278,Baixo,0,22.645279
10,Batel,1700.0,0.48,15.89,0.172879,351.0,1378.0,2685.0,10878.0,173.77,1.74,62.60,9.935571,Alto,1,246.828461


## Modelo

Nosso modelo usa apenas três variáveis independentes: renda média, taxa de analfabetismo e estabelecimentos de serviço por mil habitantes. O objetivo é identificar o quanto estas variáveis influenciam na urbanização dos bairros. O modelo é especificado e ajustado abaixo:

In [41]:
model = smf.logit("Urbanizado ~ Renda_Média + Taxa_Analfabetismo + Estabelecimentos_Ativos_Serviço_per1k", data=df)

response = model.fit()
response.summary()

Optimization terminated successfully.
         Current function value: 0.491764
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Urbanizado   No. Observations:                   50
Model:                          Logit   Df Residuals:                       46
Method:                           MLE   Df Model:                            3
Date:                Fri, 29 Oct 2021   Pseudo R-squ.:                  0.2905
Time:                        09:53:50   Log-Likelihood:                -24.588
converged:                       True   LL-Null:                       -34.657
Covariance Type:            nonrobust   LLR p-value:                 0.0001589
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                -0.3350      2.858     -0.117      0.907      -5.937       5.267
Renda_Média                              -0.0023      0.003     -0.841      0.400      -0.008       0.003
Taxa_Analfabetismo                       -0.1293      0.383     -0.338      0.736      -0.880       0.621
Estabelecimentos_Ativos_Serviço_per1k     0.0820      0.037      2.242      0.025       0.010       0.154
=========================================================================================================
"""

Nos resultados do modelo podemos perceber que apenas a variável de estabelecimentos de serviços é estatisticamente significativa (p-value < 0.05). Para interpretar o coeficiente das variáveis, precisamos calcular a razão de chance (odds ratio), como feito abaixo:

In [43]:
params = response.params
conf = response.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
print(np.exp(conf))

                                             5%         95%  Odds Ratio
Intercept                              0.002639  193.927014    0.715343
Renda_Média                            0.992348    1.003073    0.997696
Taxa_Analfabetismo                     0.414940    1.860951    0.878740
Estabelecimentos_Ativos_Serviço_per1k  1.010359    1.166201    1.085487


O odds ratio da variável de estabelecimentos é 1.08, o que significa que para um aumento de um estabelecimento por mil habitantes a chace do bairro ser considerado urbanizado aumenta em 8%.